<div style="padding:10px; color:white; margin:5; font-size:170%; text-align:left; display:fill; border-radius:15px; background-color:#294B8E; overflow:hidden; font-weight:700">
Denoising Autoencoder
</div>

In [1]:
import os, random, zipfile
from pathlib import Path
from PIL import ImageFilter

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
import torchvision.utils as vutils

# Hyperparameters 
EPOCHS = 200          
BATCH_SIZE = 64
IMGSZ = 64
LR = 1e-3
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
NUM_WORKERS = 2
SEED = 42
SAVE_EVERY = 5       # save checkpoint every n epochs
NOISE_SIGMA = 0.15   # gaussian noise std dev
BLUR_PROB = 0.15     # probability to apply blur (per image) during training

In [2]:
# Paths
INPUT_ROOT = Path("/kaggle/input/garbage-classification/Garbage classification")
OUT_ROOT = Path("/kaggle/working/denoiser_out")
CHECKPOINT_DIR = OUT_ROOT / "checkpoints"
EXAMPLES_DIR = OUT_ROOT / "denoised_examples"
for d in (OUT_ROOT, CHECKPOINT_DIR, EXAMPLES_DIR):
    d.mkdir(parents=True, exist_ok=True)

# reproducibility
random.seed(SEED)
torch.manual_seed(SEED)

In [3]:
# detect image root (handles inner folder)
img_root = None
if INPUT_ROOT.exists():
    candidate = INPUT_ROOT / "Garbage classification"
    if candidate.exists():
        img_root = candidate
    else:
        subdirs = [p for p in INPUT_ROOT.iterdir() if p.is_dir()]
        found = False
        for s in subdirs:
            if any(c.is_dir() for c in s.iterdir()):
                img_root = s
                found = True
                break
        if not found:
            img_root = INPUT_ROOT
else:
    raise FileNotFoundError(f"Dataset path not found: {INPUT_ROOT}")

print("Using image root:", img_root)

# transforms (autoencoder uses [0,1] images)
transform = transforms.Compose([
    transforms.Resize(IMGSZ),
    transforms.CenterCrop(IMGSZ),
    transforms.ToTensor()   # range [0,1]
])

dataset = datasets.ImageFolder(root=str(img_root), transform=transform)
class_names = dataset.classes
print("Classes:", class_names, "num:", len(class_names))

dataloader = DataLoader(dataset,
                        batch_size=BATCH_SIZE,
                        shuffle=True,
                        num_workers=NUM_WORKERS,
                        pin_memory=True)

Using image root: /kaggle/input/garbage-classification/Garbage classification/Garbage classification
Classes: ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash'] num: 6


In [4]:
#  model, noise helpers, init
import torch.nn.functional as F

class ConvDenoiser(nn.Module):
    def __init__(self):
        super().__init__()
        # Encoder
        self.enc = nn.Sequential(
            nn.Conv2d(3, 64, 3, stride=2, padding=1),   # 64 -> 32
            nn.ReLU(True),
            nn.Conv2d(64,128,3,stride=2,padding=1),     # 32 -> 16
            nn.ReLU(True),
            nn.Conv2d(128,256,3,stride=2,padding=1),    # 16 -> 8
            nn.ReLU(True),
        )
        # Decoder
        self.dec = nn.Sequential(
            nn.ConvTranspose2d(256,128,4,2,1),  # 8 -> 16
            nn.ReLU(True),
            nn.ConvTranspose2d(128,64,4,2,1),   # 16 -> 32
            nn.ReLU(True),
            nn.ConvTranspose2d(64,3,4,2,1),     # 32 -> 64
            nn.Sigmoid()  # output in [0,1]
        )

    def forward(self, x):
        z = self.enc(x)
        out = self.dec(z)
        return out

def add_noise(batch, noise_sigma=0.15, blur_prob=0.15):
    # batch: tensor [B, C, H, W] in [0,1]
    noisy = batch + torch.randn_like(batch) * noise_sigma
    noisy = torch.clamp(noisy, 0.0, 1.0)
    # optional blur image-wise (PIL)
    if blur_prob > 0:
        noisy_list = []
        for img in noisy:
            if random.random() < blur_prob:
                pil = transforms.ToPILImage()(img.cpu())
                pil = pil.filter(ImageFilter.GaussianBlur(radius=1.5))
                noisy_list.append(transforms.ToTensor()(pil))
            else:
                noisy_list.append(img.cpu())
        noisy = torch.stack(noisy_list).to(batch.device)
    return noisy

In [5]:
# instantiate
model = ConvDenoiser().to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=LR)
criterion = nn.MSELoss()

# resume if checkpoint exists
start_epoch = 1
ckpts = sorted(list(CHECKPOINT_DIR.glob("denoiser_epoch*.pth")), key=lambda p: p.stat().st_mtime, reverse=True)
if ckpts:
    latest = ckpts[0]
    try:
        ck = torch.load(latest, map_location=DEVICE)
        model.load_state_dict(ck["model"])
        optimizer.load_state_dict(ck["optim"])
        start_epoch = int(ck.get("epoch", 0)) + 1
        print("Resumed from", latest, "starting at epoch", start_epoch)
    except Exception as e:
        print("Could not load checkpoint:", e)
else:
    print("No checkpoint found, starting fresh")

No checkpoint found, starting fresh


In [6]:
# training loop
from tqdm import tqdm
import torch

print("Training on", DEVICE)
for epoch in range(start_epoch, EPOCHS+1):
    model.train()
    running_loss = 0.0
    loop = tqdm(dataloader, desc=f"Epoch {epoch}/{EPOCHS}")
    for imgs, _ in loop:
        imgs = imgs.to(DEVICE)               # clean images in [0,1]
        noisy = add_noise(imgs, noise_sigma=NOISE_SIGMA, blur_prob=BLUR_PROB)
        optimizer.zero_grad()
        recon = model(noisy)
        loss = criterion(recon, imgs)        # MSE between recon and clean
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * imgs.size(0)
        loop.set_postfix(loss=loss.item())
    epoch_loss = running_loss / len(dataset)
    print(f"Epoch {epoch} average loss: {epoch_loss:.6f}")

    # save checkpoint
    if epoch % SAVE_EVERY == 0:
        torch.save({"model": model.state_dict(), "optim": optimizer.state_dict(), "epoch": epoch},
                   CHECKPOINT_DIR / f"denoiser_epoch{epoch}.pth")

    # save small preview grid (first batch)
    model.eval()
    with torch.no_grad():
        sample_clean, _ = next(iter(dataloader))
        sample_noisy = add_noise(sample_clean.to(DEVICE), noise_sigma=NOISE_SIGMA, blur_prob=BLUR_PROB)
        recon = model(sample_noisy.to(DEVICE)).cpu()
    # build grid: show clean / noisy / recon (first 8)
    n = min(8, sample_clean.size(0))
    grid_clean = vutils.make_grid(sample_clean[:n], nrow=4)
    grid_noisy = vutils.make_grid(sample_noisy[:n].cpu(), nrow=4)
    grid_recon = vutils.make_grid(recon[:n], nrow=4)
    vutils.save_image(grid_clean, EXAMPLES_DIR / f"epoch{epoch}_clean.png")
    vutils.save_image(grid_noisy, EXAMPLES_DIR / f"epoch{epoch}_noisy.png")
    vutils.save_image(grid_recon, EXAMPLES_DIR / f"epoch{epoch}_recon.png")
    print(f"Saved preview images for epoch {epoch} to {EXAMPLES_DIR}")

Training on cuda


Epoch 1/200: 100%|██████████| 40/40 [00:08<00:00,  4.45it/s, loss=0.0201]

Epoch 1 average loss: 0.029242


Saved preview images for epoch 1 to /kaggle/working/denoiser_out/denoised_examples


Epoch 2/200: 100%|██████████| 40/40 [00:04<00:00,  9.00it/s, loss=0.0101] 

...

Epoch 199/200: 100%|██████████| 40/40 [00:04<00:00,  9.40it/s, loss=0.00143]

Epoch 199 average loss: 0.001344


Saved preview images for epoch 199 to /kaggle/working/denoiser_out/denoised_examples


Epoch 200/200: 100%|██████████| 40/40 [00:04<00:00,  9.81it/s, loss=0.000995]


Epoch 200 average loss: 0.001358
Saved preview images for epoch 200 to /kaggle/working/denoiser_out/denoised_examples


In [7]:
# run model on val/sample images and save denoised outputs 
from PIL import Image
model.eval()
OUT_SAMPLES_DIR = OUT_ROOT / "denoised_outputs"
OUT_SAMPLES_DIR.mkdir(parents=True, exist_ok=True)

# choose up to M images from dataset 
M = 100
count = 0
for img_path, _ in dataset.samples:
    if count >= M:
        break
    # load image using PIL and same transform pipeline (no augmentation)
    img = transforms.ToTensor()(Image.open(img_path).convert("RGB").resize((IMGSZ, IMGSZ)))
    img = img.unsqueeze(0).to(DEVICE)
    noisy = add_noise(img, noise_sigma=NOISE_SIGMA, blur_prob=BLUR_PROB)
    with torch.no_grad():
        recon = model(noisy).cpu().squeeze(0)  # [C,H,W] in [0,1]
    # save noisy and denoised side-by-side
    recon_pil = transforms.ToPILImage()(recon.clamp(0,1))
    out_path = OUT_SAMPLES_DIR / f"denoised_{count+1:04d}.png"
    recon_pil.save(out_path)
    count += 1
print(f"Saved {count} denoised outputs to {OUT_SAMPLES_DIR}")

Saved 100 denoised outputs to /kaggle/working/denoiser_out/denoised_outputs


In [9]:
# zip the outputs for download
zip_path = OUT_ROOT / "denoiser_out.zip"
with zipfile.ZipFile(zip_path, "w", compression=zipfile.ZIP_DEFLATED) as zf:
    for root, _, files in os.walk(OUT_SAMPLES_DIR):
        for f in files:
            full = Path(root) / f
            zf.write(full, arcname=str(full.relative_to(OUT_ROOT)))
print("Zipped denoised outputs to", zip_path)

Zipped denoised outputs to /kaggle/working/denoiser_out/denoiser_out.zip
